### Early stopping

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras.models import Sequential

In [2]:
data = pd.read_csv("mnist_train.csv")
data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
X  = data.drop(['label'],axis = 1)
X.head()
y = data['label']

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=10)
y_test  = to_categorical(y_test, num_classes=10)

In [32]:
model = Sequential()
model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(10,activation='softmax'))

In [33]:
from tensorflow.keras.callbacks import Callback

In [34]:
class Earlystopper(Callback):
    def __init__(self,target):
        super(Earlystopper,self).__init__()
        self.target = target
    def on_epoch_end(self,epoch,logs = {}):
        acc = logs['val_accuracy']
        if acc >= self.target:
            self.model.stop_training = True

In [35]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [36]:
early_stopper = Earlystopper(0.95)


In [37]:
history = model.fit(X_train,y_train,
                    validation_data = (X_test,y_test),
                    epochs = 10,
                    batch_size = 32,
                    callbacks = [early_stopper])

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.7866 - loss: 1.4747 - val_accuracy: 0.8966 - val_loss: 0.3841
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9122 - loss: 0.3248 - val_accuracy: 0.9254 - val_loss: 0.2872
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9324 - loss: 0.2464 - val_accuracy: 0.9392 - val_loss: 0.2271
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9445 - loss: 0.2007 - val_accuracy: 0.9457 - val_loss: 0.2037
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9513 - loss: 0.1766 - val_accuracy: 0.9446 - val_loss: 0.2054
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9570 - loss: 0.1536 - val_accuracy: 0.9538 - val_loss: 0.1788


### either val_acc>=0.95 or validation loss <0.1

In [38]:
class Earlystopper(Callback):
    def __init__(self,target1,target2):
        super(Earlystopper,self).__init__()
        self.target1 = target1
        self.target2 = target2
    def on_epoch_end(self,epoch,logs = {}):
        acc = logs['val_accuracy']
        loss = logs['val_loss']
        if acc >= self.target1 or loss < self.target2:
            self.model.stop_training = True

In [39]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [51]:
early_stopper = Earlystopper(0.98,0.3)

In [52]:
history = model.fit(X_train,y_train,
                    validation_data = (X_test,y_test),
                    epochs = 10,
                    batch_size = 32,
                    callbacks = [early_stopper])


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9917 - loss: 0.0338 - val_accuracy: 0.9632 - val_loss: 0.2929


### checkpoints

In [53]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [54]:
filepath = "best_model.keras"
check_point = ModelCheckpoint(filepath=filepath,
                              monitor = 'val_accuracy',
                              verbose = 1,
                              save_best_only = True,
                              mode = 'auto')

In [56]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [57]:
history1 = model.fit(X_train,y_train,
                    validation_data = (X_test,y_test),
                    epochs = 10,
                    batch_size = 32,
                    callbacks = [check_point])


Epoch 1/10
1499/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9923 - loss: 0.0365
Epoch 1: val_accuracy did not improve from 0.96925
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9926 - loss: 0.0347 - val_accuracy: 0.9687 - val_loss: 0.4074
Epoch 2/10
1491/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9945 - loss: 0.0211
Epoch 2: val_accuracy did not improve from 0.96925
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9940 - loss: 0.0243 - val_accuracy: 0.9674 - val_loss: 0.3732
Epoch 3/10
1489/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9927 - loss: 0.0338
Epoch 3: val_accuracy did not improve from 0.96925
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9918 - loss: 0.0377 - val_accuracy: 0.9647 - val_loss: 0.3364
Epoch 4/10
1494/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9948 - loss: 0.0192
Epoch 4: val_accuracy did not improve from 0.96925
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9947 - loss: 0.0207 - val_accu

In [66]:
class checkpoint(Callback):
    def __init__(self):
        super().__init__()
        self.best = -float('inf')
    def on_epoch_end(self,epoch,logs = {}):
        acc = logs['val_accuracy']
        if acc == None:
            return 
        if acc > self.best: 
            self.best = acc
            acc_percent = round(acc * 100,4) 
            filename = f"dnn_model_val_acc_{acc_percent}.keras"
            self.model.save(filename)
            print(f"\n✅ Epoch {epoch+1}: val_accuracy improved to {acc:.4f}. Saved as {filename}")
            

In [67]:
checkpoint = checkpoint()

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32,
    callbacks=[checkpoint]
)

Epoch 1/10
1496/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9937 - loss: 0.0305
✅ Epoch 1: val_accuracy improved to 0.9673. Saved as dnn_model_val_acc_96.7333.keras
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9950 - loss: 0.0216 - val_accuracy: 0.9673 - val_loss: 0.5332
Epoch 2/10
1488/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9954 - loss: 0.0246
✅ Epoch 2: val_accuracy improved to 0.9678. Saved as dnn_model_val_acc_96.7833.keras
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9946 - loss: 0.0333 - val_accuracy: 0.9678 - val_loss: 0.4598
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9955 - loss: 0.0192 - val_accuracy: 0.9677 - val_loss: 0.5221
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9946 - loss: 0.0217
✅ Epoch 4: val_accuracy improved to 0.9679. Saved as dnn_model_val_acc_96.7917.keras
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9948 - loss: 0.0236 - val_accuracy: 0.9679 - val_loss: 0